In [11]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
import gensim.corpora as corpora
from pprint import pprint
from gensim import  models


import pickle
from gensim.test.utils import datapath

# Import Dataset
papers = pd.read_csv('csv/testdata1.csv')
#print(papers.head())
print(papers.head())

   Unnamed: 0                                               text
0           0  32 THIS ARTICLE HAS BEEN PEER-REVIEWED.  COMPU...


In [12]:
# Remove punctuation
papers['text'] = \
papers['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['text'] = \
papers['text'].map(lambda x: x.lower())
# Print out the first rows of papers
print(papers['text'].head())


stop_words = stopwords.words('english')

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
             
data = papers.text.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

0    32 this article has been peer-reviewed  comput...
Name: text, dtype: object
['article', 'peer', 'reviewed', 'computing', 'science', 'learning', 'climate', 'informatics', 'accelerating', 'discovering', 'climate', 'science', 'machine', 'learning', 'goal', 'climate', 'informatics', 'emerging', 'discipline', 'inspire', 'collaboration', 'climate', 'scientists', 'data', 'scientists', 'order', 'develop', 'tools', 'analyze', 'complex']


<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
/var/folders/cv/d1w1tlhn18d523dngzfhkzcc0000gn/T/ipykernel_4940/343129088.py:3: DeprecationWarning: invalid escape sequence \.
  papers['text'].map(lambda x: re.sub('[,\.!?]', '', x))


In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])


[(0, 3), (1, 2), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 3), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 2), (29, 1)]


In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto')

pprint(lda_model.print_topics())

[(0,
  '0.001*"climate" + 0.001*"model" + 0.001*"data" + 0.001*"models" + '
  '0.001*"learning" + 0.001*"predictions" + 0.001*"temperature" + '
  '0.001*"algorithm" + 0.001*"learn" + 0.001*"global"'),
 (1,
  '0.001*"model" + 0.001*"climate" + 0.001*"models" + 0.001*"learning" + '
  '0.001*"predictions" + 0.001*"data" + 0.001*"algorithm" + 0.001*"future" + '
  '0.001*"scientists" + 0.001*"temperature"'),
 (2,
  '0.001*"model" + 0.001*"climate" + 0.001*"data" + 0.001*"models" + '
  '0.001*"predictions" + 0.001*"science" + 0.001*"prediction" + 0.001*"global" '
  '+ 0.001*"temperature" + 0.001*"scientists"'),
 (3,
  '0.004*"climate" + 0.002*"model" + 0.002*"data" + 0.002*"models" + '
  '0.002*"predictions" + 0.001*"prediction" + 0.001*"science" + '
  '0.001*"temperature" + 0.001*"global" + 0.001*"learning"'),
 (4,
  '0.037*"climate" + 0.023*"model" + 0.014*"data" + 0.012*"models" + '
  '0.011*"predictions" + 0.008*"learning" + 0.007*"algorithm" + 0.007*"global" '
  '+ 0.007*"science" + 0.0

In [15]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/Users/wenxiuye/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/wenxiuye/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.060131 -0.042254       1        1  99.955524
3     -0.012157  0.008655       2        1   0.012104
0     -0.015718  0.010895       3        1   0.011310
2     -0.016106  0.011248       4        1   0.010662
1     -0.016150  0.011456       5        1   0.010400, topic_info=              Term        Freq       Total Category  logprob  loglift
181        climate  118.000000  118.000000  Default  30.0000  30.0000
756          model   74.000000   74.000000  Default  29.0000  29.0000
271           data   45.000000   45.000000  Default  28.0000  28.0000
760         models   39.000000   39.000000  Default  27.0000  27.0000
926    predictions   37.000000   37.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
1318           vol    0.000254   12.161363   Topic5  -7.1848  -1.6044
759       modeling    0.000254   12.165272   Topic5  -7.1850  -1.6049
1116   simulations    0.000253    9.394502   Topic5  -7.1879  -1.3493
831   observations    0.000254   13.063416   Topic5  -7.1849  -1.6760
888    performance    0.000254   10.333952   Topic5  -7.1876  -1.4444

[375 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
2         1  0.905091         absolute
6         1  0.904784           accmip
12        1  0.902473      acknowledge
13        1  0.903249  acknowledgments
20        1  0.904846         adjacent
...     ...       ...              ...
1342      1  0.902050       widespread
1344      1  0.902647             wind
1346      1  0.904876             word
1347      1  0.960503             work
1358      1  0.901979               xp

[185 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 3, 2])